In [1]:
%reload_ext autoreload 
%autoreload 2

import pandas as pd
import numpy as np
import re
import os
import sys
import time
sys.path.append("../../")
from typing import List, Dict
from os.path import join, dirname
from pydrive.auth import GoogleAuth
from datetime import datetime
from utils import gdrive_downloader

In [2]:
GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '../../utils/client_secrets.json'
drive = gdrive_downloader.gdrive_authenticate()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=17797463459-h3beca6cno86q9eu3of312ki2g8dv7f5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
df_dict = gdrive_downloader.download_data(drive)
[{key: len(value)} for key, value in df_dict.items()]

pageviews
0.0
0.08888888888888889
0.17777777777777778
0.26666666666666666
0.35555555555555557
0.4444444444444444
0.5333333333333333
0.6222222222222222
0.7111111111111111
0.8
0.8888888888888888
0.9777777777777777
events
0.0
0.10869565217391304
0.21739130434782608
0.32608695652173914
0.43478260869565216
0.5434782608695652
0.6521739130434783
0.7608695652173914
0.8695652173913043
0.9782608695652174
commerce
0.0
0.10869565217391304
0.21739130434782608
0.32608695652173914
0.43478260869565216
0.5434782608695652
0.6521739130434783
0.7608695652173914
0.8695652173913043
0.9782608695652174


[{'pageviews': 14245014}, {'events': 4827199}, {'commerce': 67006}]

In [12]:
pageviews, events, commerce = [df for df in df_dict.values()]

Reformat pageviews dataframe

In [13]:
pageviews.rename(columns={'_article': 'is_article'}, inplace=True)
pageviews['article_id'].fillna('', inplace=True)
pageviews['is_article'] = pageviews['is_article'].astype(bool)
pageviews['article_id'] = pageviews['article_id'].astype(str)
pageviews['time'] = pd.to_datetime(pageviews['time'])
pageviews['date'] = pageviews['time'].dt.date
pageviews['remp_session_id'].fillna('', inplace = True)
pageviews['signed_in'].fillna('', inplace = True)
pageviews['social'].fillna('', inplace = True)
pageviews['tags'].fillna('', inplace = True)
pageviews['title'].fillna('', inplace = True)
pageviews['user_agent'].fillna('', inplace = True)
[pageviews[column].fillna('', inplace = True) for column in pageviews.columns if re.search('utm_', column) is not None]
pageviews.head()

,is_article,action,article_id,category,host,name,referer,remp_session_id,signed_in,social,...,title_variant,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source,date
0,True,load,6734.0,nezaradene,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,,...,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/6734/biznis-s-titulmi-kto-p...,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,,2018-02-03
1,True,load,165977.0,veda,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,,...,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/165977/zivocisne-druhy-maso...,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,,2018-02-03
2,True,load,594122.0,komentare,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,,...,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/594122/mohamedov-zivot/,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,,2018-02-03
3,True,load,808523.0,hlavna,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,,...,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/minuta/808523/,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,,2018-02-03
4,True,load,859254.0,hlavna,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,6b60d544-53b4-4524-ad7a-06ce791ffa3b,0,,...,A,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/minuta/859254/,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,,2018-02-03


Reformat commerce dataframe

In [43]:
commerce.head()

,article_id,author_id,currency,funnel_id,host,name,product_ids,revenue,step,time,token,transaction_id,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,,,,28,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00,checkout,1517616636000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,23763,,,,
1,,,,shop,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00,checkout,1517616800000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,141551,,,,
2,,,EUR,shop,npress-influxdb-n2.ewhs.bntb.net,commerce,"38,43,47",39.74,payment,1517616865000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,8577694016,https://obchod.dennikn.sk/,Mozilla/5.0 (Windows NT 5.1; rv:52.0) Gecko/20...,141551,,,,
3,,,EUR,,npress-influxdb-n2.ewhs.bntb.net,commerce,84,4.99,purchase,1517616905000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,6630687496,,,16143,,,,
4,,,,19,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00,checkout,1517616991000000000,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,133214,,,,


In [17]:
commerce_test = df_dict['commerce']

In [38]:
print(commerce.columns)
commerce['time'] = pd.to_datetime(commerce['time'].astype(int))
commerce['date'] = commerce['time'].dt.date
for column in commerce.columns:
    if column not in ['revenue']:
        commerce[column] = commerce[column].astype(str)
        commerce[column] = commerce[column].str.replace('nan', '')
    else:
        commerce['revenue'] = commerce['revenue'].astype(float)
        commerce['revenue'] = commerce['revenue'].fillna(0)
commerce.head()

Index(['article_id', 'author_id', 'currency', 'funnel_id', 'host', 'name',
       'product_ids', 'revenue', 'step', 'time', 'token', 'transaction_id',
       'url', 'user_agent', 'user_id', 'utm_campaign', 'utm_content',
       'utm_medium', 'utm_source'],
      dtype='object')


,article_id,author_id,currency,funnel_id,host,name,product_ids,revenue,step,time,token,transaction_id,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source,date
0,,,,28,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00,checkout,2018-02-03 00:10:36,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,23763,,,,,2018-02-03
1,,,,shop,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00,checkout,2018-02-03 00:13:20,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,141551,,,,,2018-02-03
2,,,EUR,shop,npress-influxdb-n2.ewhs.bntb.net,commerce,"38,43,47",39.74,payment,2018-02-03 00:14:25,1a8feb16-3e30-4f9b-bf74-20037ea8505a,8577694016,https://obchod.dennikn.sk/,Mozilla/5.0 (Windows NT 5.1; rv:52.0) Gecko/20...,141551,,,,,2018-02-03
3,,,EUR,,npress-influxdb-n2.ewhs.bntb.net,commerce,84,4.99,purchase,2018-02-03 00:15:05,1a8feb16-3e30-4f9b-bf74-20037ea8505a,6630687496,,,16143,,,,,2018-02-03
4,,,,19,npress-influxdb-n2.ewhs.bntb.net,commerce,,0.00,checkout,2018-02-03 00:16:31,1a8feb16-3e30-4f9b-bf74-20037ea8505a,,,,133214,,,,,2018-02-03


In [40]:
last_30_days = pd.Series(pageviews['date'].unique()).sort_values(ascending=False).tolist()[:30]

Comparing with funnel report built in Exponea @ https://cloud.infinario.com/p/dennikn-sk/analytics/funnels/5a76c32534e299737da62ffb

In [41]:
print('Unique visitors, last 30 days: ')
print('REMP data collection: ', len(pageviews.loc[pageviews['date'].isin(last_30_days), 'user_id'].unique()))
print('Exponea: 1126722')

Unique visitors, last 30 days: 
REMP data collection:  984895
Exponea: 1126722


In [42]:
print('Unique visitors on articles, last 30 days: ')
print('REMP data collection: ', len(df_dict['pageviews'].loc[(pageviews['date'].isin(last_30_days)) &
                                                             (pageviews['is_article'] == True), 'user_id'].unique()))
print('Exponea: 519 050')

Unique visitors on articles, last 30 days: 
REMP data collection:  844545
Exponea: 519 050


In [53]:
print('Unique visitors in sales funnel, last 30 days: ')
print('REMP data collection: ', len(commerce.loc[(commerce['date'].isin([x.isoformat() for x in last_30_days])), 'user_id'].unique()))
print('Exponea: 32')

Unique visitors in sales funnel, last 30 days: 
REMP data collection:  20290
Exponea: 32


In [59]:
print('Unique visitors signed in, last 30 days: ')
print('REMP data collection: ', len(pageviews.loc[(pageviews['date'].isin(last_30_days)) &
                                                  (pageviews['user_id'].str.match('^[0-9]+$')), 'user_id'].unique()))
print('Exponea: 67001')

Unique visitors signed in, last 30 days: 
REMP data collection:  46775
Exponea: 67001


In [61]:
print('Unique visitors on articles signed in, last 30 days: ')
print('REMP data collection: ', len(df_dict['pageviews'].loc[(pageviews['date'].isin(last_30_days)) &
                                                             (pageviews['is_article'] == True) &
                                                             (pageviews['user_id'].str.match('^[0-9]+$')), 'user_id'].unique()))
print('Exponea: 55558')

Unique visitors on articles signed in, last 30 days: 
REMP data collection:  45983
Exponea: 55558


In [62]:
print('Unique visitors in sales funnel signed in, last 30 days: ')
print('REMP data collection: ', len(commerce.loc[(commerce['date'].isin([x.isoformat() for x in last_30_days])) &
                                                 (commerce['user_id'].str.match('^[0-9]+$')), 'user_id'].unique()))
print('Exponea: 28')

Unique visitors in sales funnel signed in, last 30 days: 
REMP data collection:  19622
Exponea: 28
